In [163]:
import os

from astropy.table import Table
from astropy.io import ascii
from astropy.io import fits
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
from astropy import units as u

import numpy as np

import pandas as pd

from regions import PixCoord, PolygonSkyRegion, PolygonPixelRegion

import sfdmap

from scipy.integrate import quad, dblquad, tplquad



In [61]:
# from dustmaps.config import config
# config['data_dir'] = '/home/mj1e16/DUST_DIR/'

# import dustmaps.sfd
# dustmaps.sfd.fetch()

In [66]:
schalf_dust_map = sfdmap.SFDMap('/home/mj1e16/DUST_DIR/sfd')

In [2]:
databaseDir = '/home/mj1e16/outTablesImproved/extendedMagRange/'
imdir = r'/media/mj1e16/PP AV-TV/keplerCal/'
image = r'/media/mj1e16/PP AV-TV/keplerCal/kplr2009114174833_ffi-cal.fits'

In [3]:
dirList = os.listdir(databaseDir)
databases = [x for x in dirList if '.csv' in x]
databases = [databaseDir+x for x in databases]
databases.sort()

In [4]:
imageList = os.listdir(imdir)
imageList = [x for x in imageList if '.fits' in x]
imageList = [imdir+x for x in imageList]

In [5]:
def extractQuality(table):
    tab = ascii.read(table,format='csv')
    df = tab.to_pandas()
    dfselected = df[(df['detectMinarea']==7.0)&(df['detectThresh']==7.0)&(df['filterName']=='default.conv')]
    comp = df['completenessScore'].values[0]
    acc = df['accuracyScore'].values[0]
    return comp, acc

In [207]:
def makeImageCoordinates(image,boxlength=100.):
    hdu_list = fits.open(image)
    locList = []
    for ccd in range(1,len(hdu_list)):
        image_shape = hdu_list[ccd].data.shape
        w = WCS(hdu_list[ccd])
        xiterrations = int((image_shape[0]-(image_shape[0]%boxlength))/boxlength)
        yiterrations = int((image_shape[1]-(image_shape[1]%boxlength))/boxlength)
        xVal = boxlength/2
        yVal = boxlength/2
        wx, wy = w.wcs_pix2world(xVal, yVal, 1)
        locs = [(wx,wy)]
        for x in range(xiterrations):
            yVal = boxlength/2
            xVal += boxlength
            for y in range(yiterrations):
                yVal += boxlength 
                wx, wy = w.wcs_pix2world(xVal, yVal, 1)
                locs.append((wx,wy))
        locList.append(locs)
        
    return locList

In [11]:
def findWeight(l,b,r):
    rogrim,rtgrim,rdgrim,rzgrim,rmgrim,gamma,Rc,q = 1.,1.9,3.5,0.41,6.5,1.8,2.8,0.6
    milkyWayRadius = 15.
    scaleHeight = 3.5
    bSphere = 7.669
    ebv=[]
    Rd = 3 

    radius = r * np.cos(np.deg2rad(b))
    thickness = r * np.sin(np.deg2rad(b))
                   
    xlen = (((radius*np.sin(np.deg2rad(l)))))
    ylen = ((radius*np.cos(np.deg2rad(l)))-8.364)
    dist = ((xlen**2)+(ylen**2))**0.5
    rad = ((dist**2)+(thickness**2))**0.5
    
    Bulge = (((((dist**2)+((thickness**2)/(q**2)))**0.5)/rogrim)**(-gamma))*np.exp(-(((dist**2)+((thickness**2)/(q**2)))/rtgrim**2))
    Disc = (np.exp((-rmgrim/rdgrim)-(dist/rdgrim)-(abs(thickness)/rzgrim)))
    Sphere = (np.exp(-bSphere*((rad/Rc)**0.25)))/((rad/Rc)**(7./8.))
    
    results = {}
    results[0] = [xlen,ylen,thickness]
    results[5] = [Bulge,Disc,Sphere]
    
    return results

In [120]:
def threeDintegrate(l,b,r,dl,db,dr):
    l = 2*np.pi*(l/360.)
    dl = (2*np.pi*(dl/360.))/2.
    b = 2*np.pi*(b/360.)
    db = (2*np.pi*(db/360.))/2.
    dr = dr/2.
    # dr, dl, and db are halved as they should represent the full range, with l,b,r at the centre
    
    # limits for radius
    r1 = r - dr
    r2 = r + dr
    
    # limits for theta
    t1 = l - dl
    t2 = l + dl
    
    # limits for phi
    p1 = b - db
    p2 = b + db
    
    def diff_volume(p,t,r):
        return r**2*np.sin(p)

    volume = tplquad(diff_volume, r1, r2, lambda r:   t1, lambda r:   t2,
                                          lambda r,t: p1, lambda r,t: p2)[0]
    
    return volume

In [237]:
def findRegionProbsandCompleteness(coords,radialDistance,absMag,lumFunc,completeness,boxlength,pixel_scale=4.,R=3.1):
    weightPerAngle = []
    volumePerAngle = []

    # buldgeCumulative = 0
    # discCumulative = 0
    # sphereCumulative = 0
    dr = (max(radialDistance) - min(radialDistance))/len(radialDistance)
    dl = (boxlength*pixel_scale)/3600.
    db = (boxlength*pixel_scale)/3600.
    totStars = 0
    totStarsFound = 0
    volumeCumulative = 0
    badkeys = []
    completenessKeys = completeness.keys()
    for counter,ra_dec in enumerate(coords):
        c = SkyCoord(ra_dec[0], ra_dec[1], frame='icrs', unit='deg')
        l = c.galactic.l.deg
        b = c.galactic.b.deg
        ebv = schalf_dust_map.ebv(l,b, frame='galactic', unit='degree', interpolate=False)
        A = R * ebv
        weightPerDistance = []
        volumePerDistance = []
        for dist in radialDistance:
            results = findWeight(l,b,dist)
            weightPerSect = results[5]
            weightPerDistance.append(weightPerSect)
                    
            volumePerSect = threeDintegrate(l,b,dist,dl,db,dr)
            volumeCumulative += volumePerSect
            volumePerDistance.append(volumePerSect)
            
            for magRange in range(len(absMag)):
            
                apparentMag = absMag[magRange] + (5*(np.log10(dist)-1)) + A
                starNo = np.exp(lumFunc[magRange]) * volumePerSect 
                compKey = float(int(apparentMag-18.6))
                if compKey in completenessKeys:
                    starsFound = starNo * (1-completeness[compKey])
                else:
                    starsFound = 0
                    badkeys.append(compKey)
                totStars += starNo
                totStarsFound += starsFound
                
        weightPerAngle.append(weightPerDistance)
        volumePerAngle.append(volumePerDistance)
    
    compForwholeReg = float(totStarsFound)/float(totStars)
    
    return [volumePerAngle,weightPerAngle,totStars,compForwholeReg,badkeys,volumeCumulative]

In [ ]:
compList = {}
accList = {}

y = 0

for x in range(len(databases)):
    comp,acc = extractQuality(databases[x])
    compList[y] = comp
    accList[y] = acc
    #print(comp)
    if y >= 0:
        y +=1
    else:
        y += -1
    if y > 7:
        y = -1

In [280]:
boxlength = 100
coords = makeImageCoordinates(image,boxlength=boxlength)
radialDistance = np.linspace(0.1,130.,60)


In [281]:
absoluteMag = np.linspace(-7,18,26)
singleStars = [-7.98,-7.60,-7.27,-6.72,-6.05,-5.43,-4.8,-4.18,-3.6,-3.16,-2.89,-2.63,-2.49,
               -2.44,-2.52,-2.41,-2.32,-2.14,-1.99,-1.82,-1.9,-2.0,-2.0,-2.1,-2.1,-2.2]

In [ ]:
results = findRegionProbsandCompleteness(coords[0],radialDistance,absoluteMag,singleStars,compList,boxlength)

In [253]:
noSteps = 60
#b = [0]
bSteps = 100
lSteps = 100
rSteps = 260

b = np.linspace(-90.,90,bSteps)
l = np.linspace(0., 359., lSteps)
radialDistance = np.linspace(0.1,130.,rSteps)#noSteps)#15)



dr = (radialDistance[-1]-radialDistance[0])/rSteps
dl = (l[-1]-(l[0]))/lSteps
db =(b[-1]-(b[0]))/bSteps

l,b = [0],[0]

volumeSegment = []
#rM = 5
probabilities = []

volVal = 0

#magrange = np.linspace(13,22,20)
resultList = []
for x,angle in enumerate(l):
    resultPerB = []
    for lat,lattitude in enumerate(b):
        resultPerD = []
        for y,item in enumerate(radialDistance):
            radius = item * np.cos(np.deg2rad(lattitude))
            xlen = (((radius*np.sin(np.deg2rad(angle)))))
            ylen = ((radius*np.cos(np.deg2rad(angle)))-8.364)
            Dist = ((xlen**2)+(ylen**2))**0.5
            width = Dist
            thickness = item * np.sin(np.deg2rad(lattitude))
            distancefromcentre = ((thickness**2)+(Dist**2))**0.5
            if abs(width) < 100 and abs(thickness) < 100:# and distancefromcentre > 1.:
                result = findWeight(angle,lattitude,item)
                probabilities.append(result[5])
                volSeg = threeDintegrate(angle,lattitude,item,dl,db,dr)
                volumeSegment.append(volSeg)
                volVal += volSeg
            else:
                print(width,thickness)
                
bulgecumalative = 0
disccumalative = 0
spherecumalative = 0

bulgeProb = []
discProb = []
sphereProb = []



for x in range(len(probabilities)):
    bulgecumalative += probabilities[x][0]#*volumeSegment[x]
    disccumalative += probabilities[x][1]#*volumeSegment[x]
    spherecumalative += probabilities[x][2]#*volumeSegment[x]
    
    bulgeProb.append(probabilities[x][0])#*volumeSegment[x])
    discProb.append(probabilities[x][1])#*volumeSegment[x])
    sphereProb.append(probabilities[x][2])#*volumeSegment[x])

totcumulative = bulgecumalative + disccumalative + spherecumalative
# Relative Mass Ratios are decided Below
discnormalisation = (2.*totcumulative)/(3.8*disccumalative)
bulgenormalisation = (1.*totcumulative)/(3.8*bulgecumalative)
spherenormalisation = (0.8*totcumulative)/(3.8*spherecumalative)

discProb = [x*discnormalisation for x in discProb]
bulgeProb = [x*bulgenormalisation for x in bulgeProb]
sphereProb = [x*spherenormalisation for x in sphereProb]

totProb = [a+b+c for a,b,c in zip(discProb,bulgeProb,sphereProb)]
print(totProb)
print(sum(totProb))
totSum = sum(totProb)
totProb = [x/totSum for x in totProb]
print(sum(totProb))

(100.06959073359074, 0.0)
(100.57113513513514, 0.0)
(101.07267953667954, 0.0)
(101.57422393822394, 0.0)
(102.07576833976835, 0.0)
(102.57731274131274, 0.0)
(103.07885714285715, 0.0)
(103.58040154440155, 0.0)
(104.08194594594595, 0.0)
(104.58349034749035, 0.0)
(105.08503474903475, 0.0)
(105.58657915057915, 0.0)
(106.08812355212356, 0.0)
(106.58966795366796, 0.0)
(107.09121235521236, 0.0)
(107.59275675675676, 0.0)
(108.09430115830116, 0.0)
(108.59584555984557, 0.0)
(109.09738996138996, 0.0)
(109.59893436293437, 0.0)
(110.10047876447877, 0.0)
(110.60202316602317, 0.0)
(111.10356756756757, 0.0)
(111.60511196911197, 0.0)
(112.10665637065638, 0.0)
(112.60820077220077, 0.0)
(113.10974517374518, 0.0)
(113.61128957528958, 0.0)
(114.11283397683398, 0.0)
(114.61437837837839, 0.0)
(115.11592277992278, 0.0)
(115.61746718146719, 0.0)
(116.11901158301158, 0.0)
(116.62055598455599, 0.0)
(117.12210038610039, 0.0)
(117.62364478764479, 0.0)
(118.1251891891892, 0.0)
(118.6267335907336, 0.0)
(119.128277992

In [16]:
print(disccumalative*discnormalisation)
print(bulgecumalative*bulgenormalisation)
print(spherecumalative*spherenormalisation)

15.998514687244192
7.999257343622095
6.399405874897678


In [92]:
weightPerAngle[0][0]

{0: [0.09786550400003544, -8.346759628422824, 0.011185379506297537],
 5: [9.092606473209177e-11, 0.01399050135190474, 1.6161165872166317e-05]}

In [20]:
keplerBtot = 0
keplerDtot = 0
keplerStot = 0

for ang in range(len(weightPerAngle)):
    for dist in range(len(weightPerAngle[ang])):
        keplerBtot += weightPerAngle[ang][dist][0]
        keplerDtot += weightPerAngle[ang][dist][1]
        keplerStot += weightPerAngle[ang][dist][2]
        
normKeplerBtot = keplerBtot*bulgenormalisation
normKeplerDtot = keplerDTot*discnormalisation
normKeplerStot = keplerStot*spherenormalisation

In [23]:
normKeplerStot

0.17667814884443328

In [24]:
normKeplerStot/31.

0.005699295124013977

In [27]:
print(sum(sphereProb))

6.39940587489768


In [ ]:
# need to test half total galaxy with different spacing

In [ ]:
ebv = schalf_dust_map.ebv(l,b, frame='galactic', unit='degree', interpolate=False)
R = 3.1
reddening = ((ebv)*R)

In [42]:
absoluteMag = np.linspace(-7,18,26)

In [43]:
absoluteMag

array([-7., -6., -5., -4., -3., -2., -1.,  0.,  1.,  2.,  3.,  4.,  5.,
        6.,  7.,  8.,  9., 10., 11., 12., 13., 14., 15., 16., 17., 18.])

In [46]:
singleStars = [-7.98,-7.60,-7.27,-6.72,-6.05,-5.43,-4.8,-4.18,-3.6,-3.16,-2.89,-2.63,-2.49,
               -2.44,-2.52,-2.41,-2.32,-2.14,-1.99,-1.82,-1.9,-2.0,-2.0,-2.1,-2.1,-2.2]

print(len(singleStars))

26


In [50]:
float(int(11.7))

11.0

In [53]:
a = {}
a[1] = 0

In [55]:
print(a.keys())

[1]


In [164]:
polygon_pix = PolygonPixelRegion(
    vertices=PixCoord(x=[0, 0, 100, 100], y=[0, 100, 0,100]),
)


In [165]:
polygon_pix

<PolygonPixelRegion(vertices=PixCoord(x=[  0   0 100 100], y=[  0 100   0 100]))>

In [166]:
sky_reg = PixCoord.from_sky(skycoord=polygon_pix,wcs=)

In [168]:
print(imageList[0])

/media/mj1e16/PP AV-TV/keplerCal/kplr2009114174833_ffi-cal.fits


In [169]:
imageFile = fits.open(imageList[0])

In [208]:
w = WCS(imageFile[-1])

In [173]:
sky_reg = polygon_pix.from_sky(wcs=w)

AttributeError: 'PolygonPixelRegion' object has no attribute 'from_sky'

In [174]:
pixcoord = PixCoord(x=[0, 0, 100, 100], y=[0, 100, 0,100])

In [175]:
sky_reg = pixcoord.to_sky(wcs=w)

In [176]:
sky_reg

<SkyCoord (ICRS): (ra, dec) in deg
    [(282.13271338, 45.1517173 ), (282.01100839, 45.08241187),
     (282.23093618, 45.06584791), (282.10929388, 44.99663584)]>

In [189]:
print(ra)

[282.13271338 282.01100839 282.23093618 282.10929388]


In [190]:
print(dec)

[45.1517173  45.08241187 45.06584791 44.99663584]


In [202]:
ra = sky_reg.ra.arcsec
dec = sky_reg.dec.arcsec

In [201]:
ra

array([1015677.76815635, 1015458.57409509, 1015854.69654538,
       1015635.55902744])

In [234]:
def findSideLengths(center,boxlength,wcs):
    d_box = boxlength/2.
    
    minx = centre[0] - d_box
    maxx = centre[0] + d_box

    miny = centre[1] - d_box
    maxy = centre[1] + d_box
    
    pixcoord = PixCoord(x=[minx, minx, maxx, maxx], y=[miny, maxy, miny,maxy])
    sky_reg = pixcoord.to_sky(wcs=w)
    
    ra = sky_reg.galactic.l.arcsec
    dec = sky_reg.galactic.b.arcsec

    xlengths = []
    ylengths = []

    xlengths.append(abs(ra[0]-ra[1]))
    ylengths.append(abs(dec[0]-dec[1]))

    xlengths.append(abs(ra[0]-ra[2]))
    ylengths.append(abs(dec[0]-dec[2]))

    xlengths.append(abs(ra[1]-ra[3]))
    ylengths.append(abs(dec[1]-dec[3]))

    xlengths.append(abs(ra[2]-ra[3]))
    ylengths.append(abs(dec[2]-dec[3]))


    distances = [((x**2)+(y**2))**0.5 for x,y in zip(xlengths,ylengths)]
    
    db = np.mean([distances[0],distances[-1]])
    dl = np.mean([distances[1],distances[2]])
    
    return db, dl

In [235]:
w = WCS(imageFile[-1])

minx = 2000
maxx = 2050

miny = 2000
maxy = 2050

pixcoord = PixCoord(x=[minx, minx, maxx, maxx], y=[miny, maxy, miny,maxy])
sky_reg = pixcoord.to_sky(wcs=w)

ra = sky_reg.galactic.l.arcsec
dec = sky_reg.galactic.b.arcsec

xlengths = []
ylengths = []

xlengths.append(abs(ra[0]-ra[1]))
ylengths.append(abs(dec[0]-dec[1]))

xlengths.append(abs(ra[0]-ra[2]))
ylengths.append(abs(dec[0]-dec[2]))

xlengths.append(abs(ra[1]-ra[3]))
ylengths.append(abs(dec[1]-dec[3]))

xlengths.append(abs(ra[2]-ra[3]))
ylengths.append(abs(dec[2]-dec[3]))


distances = [((x**2)+(y**2))**0.5 for x,y in zip(xlengths,ylengths)]

#distances_pix = [(x) for x in distances]
print(distances)
print(sum(distances))

[201.09437507529378, 199.9348927639803, 199.93875384126264, 201.11298730859505]
802.0810089891318


In [232]:
print(distances)
print(sum(distances))

[201.30188445473587, 204.6710217638068, 204.63985012966378, 201.31156419943568]
811.9243205476421


In [215]:
ylengths

[124.6852576690435, 154.51998340411228, 154.4359739701904, 124.6012482351216]

In [210]:
print(distances_pix)

[265.00722194747135, 214.99258470543515, 215.03330604639567, 265.19919522261097]


In [216]:
angle = np.tan(xlengths[1]/ylengths[1])

In [219]:
angle_deg = (angle*180.)/ np.pi
print(angle_deg)

126.3358768160967


In [188]:
for x in range(len(ra)-1):
    x1 = abs(ra[x]-ra[x+1])
    y1 = abs(dec[x]-dec[x+1])
    length = (x1**2 + y1**2)**0.5
    length_arsec = length*3600
    print(length_arsec/3.977)

126.77829429312646
199.64355679666244
126.68717745145602


In [180]:
x1 = abs(ra[0]-ra[1])
y1 = abs(dec[0]-dec[1])

In [181]:
x1

0.1217049890283306

In [182]:
length = (x1**2 + y1**2)**0.5
print(length)

0.14005479900104553


In [183]:
length_arsec = length*3600

In [184]:
length_arsec

504.1972764037639

In [185]:
length_arsec/3.977

126.77829429312646

In [205]:
len(coords)

111

In [206]:
len(coords[0])

2